In [5]:
!pip install lingam

# CSSE Funcionando

In [2]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [7]:

# Read Dataset German
df_main = prepare_german_dataset("german_credit.csv", "data/")

#Get the input features
columns = df_main.columns
class_name = 'default' # default = 0 = "Good class" / default = 1 = "Bad class" 
columns_tmp = list(columns)
columns_tmp.remove(class_name)

x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

model = RandomForestClassifier(n_estimators = 120, n_jobs=-1, random_state=0)  
model.fit(x_train, y_train)

p = model.predict(x_test)

print(classification_report(y_test, p))

#-------Begin Parameter Adjustment--------

X = 0 #Indicates the instance's position to be explained in the dataset

#User preferences
#static_list = [] #List of features that cannot be changed. For example: static_list = ['age']
K = 5 #Number of counterfactual explanations to be obtained

#Genetic Algorithm parameters
#num_gen = 30 #number of generations
#pop_size = 150 #population size
#per_elit = 0.1 #percentage of elitism
#cros_proba = 0.8 #crossover probability
#mutation_proba = 0.1 #mutation probability

#Weights of objective function metrics
#L1 = 1 #lambda 1 - Weight assigned the distance to the original instance
#L2 = 1 #lambda 2 - Weight assigned the amount of changes needed in the original instance

#copy the original instance
original_instance = x_test.iloc[X].copy() 

#-------End Parameter Adjustment--------

print('Original instance - Class ' + str(p[X]) + '\n')
print(original_instance)
print('\nGetting counterfactuals...\n')

#Run CSSE - Method executed with default parameters except for the value of K.
explainerCSSE = CSSE(df_main[columns_tmp], model, K = K, num_gen = 3)

contrafactual_set, solution = explainerCSSE.explain(original_instance, p[X]) #Method returns the list of counterfactuals and the explanations generated from them

#The method returns a list of counterfactual solutions, where each solution, in turn, is a change list (each change has the "column" and "value" to be changed). To implement another output format, see the "printResults" function
explainerCSSE.printResults(solution)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        82
           1       0.36      0.28      0.31        18

    accuracy                           0.78       100
   macro avg       0.60      0.58      0.59       100
weighted avg       0.76      0.78      0.77       100

Original instance - Class 0

account_check_status             3
duration_in_month               18
credit_history                   2
purpose                          4
credit_amount                 2100
savings                          1
present_emp_since                3
installment_as_income_perc       4
personal_status_sex              3
other_debtors                    2
present_res_since                2
property                         1
age                             37
other_installment_plans          3
housing                          2
credits_this_2                   1
job                              3
people_under_maintenance         1
telephone     

Processing...:  33%|███▎      | 1/3 [00:00<00:01,  1.20it/s]

Processing...:  67%|██████▋   | 2/3 [00:01<00:00,  1.25it/s]

Processing...: 100%|██████████| 3/3 [00:02<00:00,  1.29it/s]


Result obtained


         Counterfactual 1         
account_check_status              1


         Counterfactual 2         
duration_in_month                48
age                              21


# Testando Lingam

In [8]:
import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

In [9]:
model_lingam = lingam.DirectLiNGAM()

In [10]:
result = model_lingam.bootstrap(df_main[columns_tmp], n_sampling=100)

In [11]:
causal_effects = result.get_total_causal_effects(min_causal_effect=0.01)

# Assign to pandas.DataFrame for pretty display
df_causal_effects = pd.DataFrame(causal_effects)
labels = [f'{i}' for i in df_main[columns_tmp].columns]
df_causal_effects['from'] = df_causal_effects['from'].apply(lambda x : labels[x])
df_causal_effects['to'] = df_causal_effects['to'].apply(lambda x : labels[x])
df_causal_effects.head()

,from,to,effect,probability
0,other_debtors,other_installment_plans,1.320330,1.0
1,foreign_worker,duration_in_month,-7.274988,1.0
2,foreign_worker,age,-0.367341,1.0
3,other_debtors,telephone,0.857678,1.0
4,telephone,job,0.254906,1.0


In [31]:
df_causal_effects[df_causal_effects['probability'] == 1]

,from,to,effect,probability
0,other_debtors,other_installment_plans,1.320330,1.0
1,foreign_worker,duration_in_month,-7.274988,1.0
2,foreign_worker,age,-0.367341,1.0
3,other_debtors,telephone,0.857678,1.0
4,telephone,job,0.254906,1.0
5,people_under_maintenance,age,2.226039,1.0


In [12]:
causal_order = [labels[x] for x in model_lingam.causal_order_]
causal_order

['foreign_worker',
 'other_debtors',
 'people_under_maintenance',
 'savings',
 'other_installment_plans',
 'credits_this_2',
 'telephone',
 'credit_amount',
 'property',
 'housing',
 'personal_status_sex',
 'credit_history',
 'job',
 'account_check_status',
 'age',
 'present_res_since',
 'installment_as_income_perc',
 'duration_in_month',
 'present_emp_since',
 'purpose']

In [28]:
original_ind

account_check_status             3
duration_in_month               18
credit_history                   2
purpose                          4
credit_amount                 2100
savings                          1
present_emp_since                3
installment_as_income_perc       4
personal_status_sex              3
other_debtors                    2
present_res_since                2
property                         1
age                             37
other_installment_plans          3
housing                          2
credits_this_2                   1
job                              3
people_under_maintenance         1
telephone                        0
foreign_worker                   1
Name: 37, dtype: int64

In [ ]:
[('duration_in_month', 41),
  ('personal_status_sex', 2),
  ('age', 21),
  ('other_installment_plans', 4)]

In [27]:
pd.DataFrame(result.get_paths(2, 12))

,path,effect,probability
0,"[2, 6, 12]",0.167953,0.56
1,"[2, 12]",0.988314,0.34
2,"[2, 0, 6, 12]",0.023346,0.22
3,"[2, 6, 10, 12]",0.055522,0.13
4,"[2, 11, 16, 6, 12]",-0.003195,0.12
...,...,...,...
121,"[2, 11, 0, 6, 12]",0.001283,0.01
122,"[2, 7, 6, 10, 12]",0.003876,0.01
123,"[2, 0, 11, 12]",0.014329,0.01
124,"[2, 8, 6, 12]",0.009772,0.01


In [25]:
columns_tmp

['account_check_status',
 'duration_in_month',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings',
 'present_emp_since',
 'installment_as_income_perc',
 'personal_status_sex',
 'other_debtors',
 'present_res_since',
 'property',
 'age',
 'other_installment_plans',
 'housing',
 'credits_this_2',
 'job',
 'people_under_maintenance',
 'telephone',
 'foreign_worker']

# Apply Causality

In [71]:
def apply_causality(df):
    print(f'df len = {df.shape}')
    df_apply_causal = pd.DataFrame(columns = df.columns)
    original = df.iloc[0]
    df_apply_causal.loc[0] = original
    for index, df_row in df.iloc[1:].iterrows():
        causal_ind = df_row.copy()
        for column in causal_order:
            value_diff = causal_ind[column] - original[column]
#             print(f"value_diff = {value_diff}")
            if value_diff != 0:
                tmp_effects = df_causal_effects[df_causal_effects['from'] == column]
                for index, row in tmp_effects.iterrows():
                    prob = rnd.random()
                    if row['probability'] >= prob:
                        causal_ind[row['to']] = causal_ind[row['to']] + (value_diff * row['effect'])
        df_apply_causal.loc[len(df_apply_causal)] = causal_ind
    
    print(f'df_apply_causal len = {df_apply_causal.shape}')
    return df_apply_causal
    


In [66]:
df_apply_causal[column]

NameError: name 'df_apply_causal' is not defined

# CSSE Explain

In [62]:
from tqdm import tqdm
import random as rnd


In [72]:
original_ind = x_test.iloc[X].copy() #Original instance
#self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
current_class = p[X] #Original instance class

ind_cur_class = explainerCSSE.getBadClass()

#Gets the valid values range of each feature
features_range = []
features_range = explainerCSSE.getFeaturesRange()

#The DataFrame df will have the current population
df = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)

#Generates the initial population with popinitial mutants        
explainerCSSE.getPopInicial(df, features_range)
df_causal = df.copy()
dict_dfs = {}

# for g in tqdm(range(explainerCSSE.num_gen), desc= "Processing..."):
for g in tqdm(range(30), desc= "Processing..."):
    
    #To use on the parents of each generation
    parents = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
    
    #Copy parents to the next generation
    parents = df.copy()
    parents_causal = df_causal.copy()
#     print(f"parents = {len(parents)}")
    dict_dfs[g] = {}
    
    dict_dfs[g]['original_parents'] = parents
    dict_dfs[g]['causal_parents'] = apply_causality(parents_causal)
#     raise KeyboardInterrupt("Execução interrompida pelo usuário")
    #df will contain the new population
    df = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
    df_causal = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
    evaluation = []
    evaluation_causal = []

    #Assessing generation counterfactuals
    explainerCSSE.fitness(parents, evaluation, ind_cur_class)
    explainerCSSE.fitness(dict_dfs[g]['causal_parents'], evaluation_causal, ind_cur_class)
    
#     print(len(df))
#     dict_dfs[g]['original'] = df.copy()
#     df_causal_applied = apply_causality(df)
#     dict_dfs[g]['causal'] = df_causal_applied
#     print(f"df depois do fitness = {len(df)}")
#     raise KeyboardInterrupt("Execução interrompida pelo usuário")
    
    
    #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
    df.loc[0] = original_ind.copy()
    df_causal.loc[0] = original_ind.copy()
    
    #Copies to the next generation the per_elit best individuals
    explainerCSSE.elitism(evaluation, df, parents)
    explainerCSSE.elitism(evaluation_causal, df_causal, parents_causal)
#     raise KeyboardInterrupt("Execução interrompida pelo usuário")
#     print(f"df depois do elitism = {len(df)}")
    number_cross_repetitions = 0
    while len(df) < explainerCSSE.pop_size + 1: #+1, as the 1st position is used to store the reference individual
        number_cross_repetitions = explainerCSSE.crossover(df, parents, evaluation, number_cross_repetitions)
        number_cross_repetitions_causal = explainerCSSE.crossover(df_causal, parents_causal, evaluation_causal, number_cross_repetitions)

        mutation_op = rnd.random()
        if mutation_op <= explainerCSSE.mutation_proba:
            explainerCSSE.mutation(df, len(df) - 1, features_range)
            explainerCSSE.mutation(df_causal, len(df_causal) - 1, features_range)
#     raise KeyboardInterrupt("Execução interrompida pelo usuário")

# raise KeyboardInterrupt("Execução interrompida pelo usuário")

evaluation = []
evaluation_causal = []

#Evaluating the latest generation
explainerCSSE.fitness(df, evaluation, ind_cur_class)
explainerCSSE.fitness(df_causal, evaluation_causal, ind_cur_class)

#Order the last generation by distance to the original instance     
evaluation.sort(key=lambda individual: individual.aval_norm)
evaluation_causal.sort(key=lambda individual: individual.aval_norm) 

#Getting the counterfactual set
contrafactual_set = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
contrafactual_set, solution_list = explainerCSSE.getContrafactual(df, evaluation)

#Getting the counterfactual CAUSAL set
contrafactual_set_causal = pd.DataFrame(columns=explainerCSSE.input_dataset.columns)
contrafactual_set_causal, solution_list_causal = explainerCSSE.getContrafactual(df_causal, evaluation_causal) 

dict_dfs['contrafactual_set_causal'] = contrafactual_set_causal
dict_dfs['solution_list_causal'] = solution_list_causal

# return contrafactual_set, solution_list

Processing...:   0%|          | 0/30 [00:00<?, ?it/s]

df len = (74, 20)
df_apply_causal len = (74, 20)


Processing...:   3%|▎         | 1/30 [00:01<00:41,  1.42s/it]

df len = (67, 20)
df_apply_causal len = (67, 20)


Processing...:   7%|▋         | 2/30 [00:03<00:44,  1.57s/it]

...

Processing...:  93%|█████████▎| 28/30 [00:55<00:03,  1.92s/it]

df len = (73, 20)
df_apply_causal len = (73, 20)


Processing...:  97%|█████████▋| 29/30 [00:57<00:01,  1.94s/it]

df len = (79, 20)
df_apply_causal len = (79, 20)


Processing...: 100%|██████████| 30/30 [00:59<00:00,  1.98s/it]


In [73]:
solution_list

[[('account_check_status', 2), ('duration_in_month', 49)],
 [('account_check_status', 2), ('property', 2)],
 [('account_check_status', 2), ('housing', 1)],
 [('account_check_status', 2), ('credit_history', 0)],
 [('account_check_status', 2), ('present_emp_since', 1)]]

In [74]:
solution_list_causal

[[('account_check_status', 1)],
 [('account_check_status', 2), ('duration_in_month', 40)],
 [('account_check_status', 2), ('credit_history', 0)],
 [('duration_in_month', 50),
  ('credit_amount', 13914),
  ('present_emp_since', 1)]]

In [75]:
df_causal_effects[df_causal_effects['from'] == "account_check_status"]

,from,to,effect,probability
109,account_check_status,age,0.116446,0.54
114,account_check_status,duration_in_month,-0.288460,0.51
120,account_check_status,present_emp_since,0.080277,0.46
151,account_check_status,present_res_since,-0.080362,0.31
155,account_check_status,purpose,0.158491,0.30
165,account_check_status,credit_history,0.140635,0.24
171,account_check_status,credit_amount,-94.168334,0.21
185,account_check_status,installment_as_income_perc,0.001350,0.17
193,account_check_status,property,-0.067760,0.11
198,account_check_status,job,0.007994,0.10


In [89]:
lista_de_listas = [[t.column for t in sublist] for sublist in solution_list_causal]
lista_de_listas

[['account_check_status'],
 ['account_check_status', 'duration_in_month'],
 ['account_check_status', 'credit_history'],
 ['duration_in_month', 'credit_amount', 'present_emp_since']]

In [93]:
import pandas as pd

lista_solution_causal = [[t.column for t in sublist] for sublist in solution_list_causal]

# Inicializa uma lista para armazenar os resultados
resultados = []

# Loop sobre os valores na lista
for lista_valores in lista_solution_causal:
    if len(lista_valores) > 1:
        for v1 in lista_valores:
            for v2 in lista_valores:
                if v1 != v2:
                    # Cria uma condição para cada par de valores diferentes na lista
                    condicao = (df_causal_effects['to'].isin([v1, v2])) & (df_causal_effects['from'].isin([v1, v2]))
                    # Realiza a busca no DataFrame usando a condição e armazena os resultados
                    resultados.append(df_causal_effects[condicao])

# Concatena os resultados em um único DataFrame
resultado_final = pd.concat(resultados)

resultado_final = resultado_final.drop_duplicates()


In [96]:
causal_order

['foreign_worker',
 'other_debtors',
 'people_under_maintenance',
 'savings',
 'other_installment_plans',
 'credits_this_2',
 'telephone',
 'credit_amount',
 'property',
 'housing',
 'personal_status_sex',
 'credit_history',
 'job',
 'account_check_status',
 'age',
 'present_res_since',
 'installment_as_income_perc',
 'duration_in_month',
 'present_emp_since',
 'purpose']

In [91]:
solution_list_causal

[[('account_check_status', 1)],
 [('account_check_status', 2), ('duration_in_month', 40)],
 [('account_check_status', 2), ('credit_history', 0)],
 [('duration_in_month', 50),
  ('credit_amount', 13914),
  ('present_emp_since', 1)]]

In [100]:
resultado_final[resultado_final['from'] == "present_emp_since"]

,from,to,effect,probability
95,present_emp_since,duration_in_month,0.202277,0.62
153,present_emp_since,credit_amount,-61.721313,0.30


In [45]:
original_ind

account_check_status             3
duration_in_month               18
credit_history                   2
purpose                          4
credit_amount                 2100
savings                          1
present_emp_since                3
installment_as_income_perc       4
personal_status_sex              3
other_debtors                    2
present_res_since                2
property                         1
age                             37
other_installment_plans          3
housing                          2
credits_this_2                   1
job                              3
people_under_maintenance         1
telephone                        0
foreign_worker                   1
Name: 37, dtype: int64

In [19]:
explainerCSSE.printResults(solution_list)

Result obtained


         Counterfactual 1         
account_check_status              2
purpose                           3


         Counterfactual 2         
purpose                           0
age                              21


         Counterfactual 3         
account_check_status              2
age                              22


         Counterfactual 4         
account_check_status              1


         Counterfactual 5         
present_emp_since                 1
age                              21


In [20]:
explainerCSSE.printResults(solution_list_causal)

Result obtained


         Counterfactual 1         
duration_in_month                23
present_emp_since                 1
age                              21


         Counterfactual 2         
credit_amount                 15928
age                              42
job                               4


         Counterfactual 3         
present_emp_since                 1
age                              21
other_installment_plans           4
job                               2


         Counterfactual 4         
duration_in_month                41
personal_status_sex               2
age                              21
other_installment_plans           4


         Counterfactual 5         
credit_amount                 15928
personal_status_sex               2
age                              21


# Check causality distance

In [125]:
def euclidean_distance(series1, series2):
    # Verificando se as séries têm o mesmo comprimento
    if len(series1) != len(series2):
        raise ValueError("As séries devem ter o mesmo comprimento.")
    
    # Calculando a diferença quadrática entre os elementos das séries
    squared_diff = (series1 - series2) ** 2
    
    # Calculando a soma das diferenças quadráticas
    sum_squared_diff = np.sum(squared_diff)
    
    # Calculando a raiz quadrada da soma
    euclidean_dist = np.sqrt(sum_squared_diff)
    
    return euclidean_dist

# Check awnsers

In [ ]:
for contrafac in solution_list_causal:
    for attr in contrafac:
        

# Bases de testes

In [20]:
df_main = pd.read_csv("data/mushrooms_processada_cg.csv")
df_main

,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,veil-type,cap-shape=b,cap-shape=c,cap-shape=f,cap-shape=k,...,population=v,population=y,habitat=d,habitat=g,habitat=l,habitat=m,habitat=p,habitat=u,habitat=w,class
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,p
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,e
2,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,e
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,p
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,e
8120,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,e
8121,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,e
8122,0,1,0,1,1,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,p


In [22]:
df_main = pd.read_csv("data/heloc_dataset_v1_nm.csv")
columns = df_main.columns
class_name = 'RiskPerformance' # default = 0 = "Good class" / default = 1 = "Bad class" 
columns_tmp = list(columns)
columns_tmp.remove(class_name)

x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

dfx_full = pd.concat([x_train, x_test])
dfy_full = pd.concat([y_train, y_test])

In [23]:
df_main

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,Good,73,131,5,57,21,0,0,95,80,...,19,7,0,0,26,-8,5,2,0,100
10455,Bad,65,147,39,68,11,0,0,92,28,...,42,1,1,1,86,53,2,2,1,80
10456,Bad,74,129,6,64,18,1,1,100,-7,...,33,3,4,4,6,-8,5,-8,0,56
10457,Bad,72,234,12,113,42,2,2,96,35,...,20,6,0,0,19,-8,4,1,0,38
